In [1]:
import os

In [2]:
os.chdir("../")
%pwd

'c:\\Projects\\Chicken_disease_classification'

In [3]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [4]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [5]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])


    
    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])

        prepare_callback_config = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )

        return prepare_callback_config

In [6]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [17]:
class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config


    
    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}",
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    

    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=str(self.config.checkpoint_model_filepath),
            save_best_only=True
        )


    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]

In [22]:

try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callback_config()
    # prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    # callback_list = prepare_callbacks.get_tb_ckpt_callbacks()
    
except Exception as e:
    raise e

[2024-11-20 12:30:36,721: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-11-20 12:30:36,725: INFO: common: yaml file: params.yaml loaded successfully]
[2024-11-20 12:30:36,727: INFO: common: created directories: artifacts]
[2024-11-20 12:30:36,728: INFO: common: created directories: artifacts\prepare_call_backs\checkpoint_dir]
[2024-11-20 12:30:36,730: INFO: common: created directories: artifacts\prepare_call_backs\tensorboard_log_dir]


In [ ]:
def create_directories(directory_paths:list, verbase=True):
    """Create list of directories"""

    for path in directory_paths:
        os.makedirs("path", exist_ok=True)
        # if verbase:
        #     logger.info(f"created directories: {path}")



model_ckpt_dir = os.path.dirname("artifacts/prepare_call_backs/checkpoint_dir/model.h5")
create_directories([Path(model_ckpt_dir)])

[2024-11-20 12:34:00,451: INFO: common: created directories: artifacts\prepare_call_backs\checkpoint_dir]
